In [9]:
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
import gradio as gr

# Initialize
load_dotenv()
openai = OpenAI(api_key=os.getenv('OPENAI_API_KEY'))
MODEL = "gpt-4o-mini"

# System message
system_message = "You are a helpful assistant for FlightAI. Give short, accurate answers. If unsure, say so. "
system_message += "When booking a flight, ask for the user's departure city, destination city, name, email, and phone number one at a time, in that order."

# Ticket prices
ticket_prices = {"london": "$799", "paris": "$899", "tokyo": "$1400", "berlin": "$499"}

def get_ticket_price(city):
    return ticket_prices.get(city.lower(), "Unknown")

# Function definition for OpenAI
price_function = {
    "name": "get_ticket_price",
    "description": "Get return ticket price to a destination city.",
    "parameters": {
        "type": "object",
        "properties": {
            "destination_city": {"type": "string", "description": "Destination city"}
        },
        "required": ["destination_city"]
    }
}
tools = [{"type": "function", "function": price_function}]

# Store booking info
booking_info = {}

def generate_ticket(booking_info):
    """Generate a formatted ticket with booking details."""
    ticket = "===== FlightAI Ticket =====\n"
    ticket += f"Booking Status: CONFIRMED\n"
    ticket += f"Passenger Name: {booking_info['name']}\n"
    ticket += f"From: {booking_info['departure']}\n"
    ticket += f"To: {booking_info['destination']}\n"
    ticket += f"Ticket Price: {booking_info['price']}\n"
    ticket += f"Email: {booking_info['email']}\n"
    ticket += f"Phone: {booking_info['phone']}\n"
    ticket += "=========================\n"
    ticket += "Thank you for choosing FlightAI!"
    return ticket

def chat(message, history):
    global booking_info
    messages = [{"role": "system", "content": system_message}] + history + [{"role": "user", "content": message}]
    
    # Check if collecting booking info
    if booking_info.get("booking_started"):
        if not booking_info.get("departure"):
            booking_info["departure"] = message
            return "Please provide your destination city."
        elif not booking_info.get("destination"):
            booking_info["destination"] = message
            price = get_ticket_price(message)
            if price == "Unknown":
                booking_info = {}  # Reset if invalid destination
                return "Sorry, we don't have flights to that destination."
            booking_info["price"] = price
            return "Please provide your full name."
        elif not booking_info.get("name"):
            booking_info["name"] = message
            return "Please provide your email address."
        elif not booking_info.get("email"):
            booking_info["email"] = message
            return "Please provide your phone number."
        elif not booking_info.get("phone"):
            booking_info["phone"] = message
            ticket = generate_ticket(booking_info)
            booking_info = {}  # Reset for next booking
            return ticket

    response = openai.chat.completions.create(model=MODEL, messages=messages, tools=tools)
    
    if response.choices[0].finish_reason == "tool_calls":
        tool_call = response.choices[0].message.tool_calls[0]
        args = json.loads(tool_call.function.arguments)
        city = args.get('destination_city')
        price = get_ticket_price(city)
        
        messages.append(response.choices[0].message)
        messages.append({
            "role": "tool",
            "content": json.dumps({"destination_city": city, "price": price}),
            "tool_call_id": tool_call.id
        })
        response = openai.chat.completions.create(model=MODEL, messages=messages)
        
        # Start booking process if price is found and user wants to book
        if price != "Unknown" and "book" in message.lower():
            booking_info["booking_started"] = True
            return "Please provide your departure city."
    
    return response.choices[0].message.content

# Launch interface
gr.ChatInterface(fn=chat, type="messages").launch()

* Running on local URL:  http://127.0.0.1:7869

To create a public link, set `share=True` in `launch()`.
